In [218]:
import pickle
import os
import numpy as np

features = pickle.load(open('features_513_corrupt.pkl', 'rb'))

In [219]:
len(features)

513

In [220]:
import pandas as pd

labels = []
all_labels = pd.read_csv('labels.csv')

for name, _ in features:
    name = name.split('.ply')[0]
    # name.replace("_mirrored", "")
    if name in all_labels['name'].values:
        label = all_labels[all_labels['name'] == name]['class'].values[0]
        labels.append((name, label))

# convert labels to a DataFrame
labels = pd.DataFrame(labels, columns=['name', 'class'])
len(all_labels)

513

In [221]:
# working encoding
# # # drop K4
# labels = labels[labels['class'] != 'K4']
labels = labels[labels['class'] != 'K3B']
# labels = labels[labels['class'] != 'K3']
# labels = labels[labels['class'] != 'K6']
# labels = labels[labels['class'] != 'K1']
# labels = labels[labels['class'] != 'K2']
labels = labels[labels['class'] != 'K5']

unique_labels = labels['class'].unique()
unique_labels.sort()
encoding = {}
for i, label in enumerate(unique_labels):
    encoding[label] = i
# encoding["K2"] = encoding["K1"]
# # # encoding["K3"] = 1
# encoding["K3B"] = encoding["K3"]
# # encoding["K4"] = encoding["K3"]
# encoding["K5"] = 1
# encoding["K6"] = 3
#encoding["K5"] = encoding["K4"]
# encoding["K5"] = encoding["K4"]
# encoding["K5"] = encoding["K3"]
# encoding["K5"] = 1

labels['class']  = labels['class'].map(encoding)

In [222]:
encoding # Counter({np.int64(0): 137, np.int64(2): 125, np.int64(1): 86})


{'K1': 0, 'K2': 1, 'K3': 2, 'K4': 3, 'K6': 4}

In [223]:
print(labels['class'].unique())

[0 3 4 1 2]


In [224]:
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = []
y = []

for name, feature in features:
    # remove .ply from the end of name
    # old_name = name
    if name.endswith('.ply'):
        name = name[:-4]
    # name = name.replace("_mirrored", "")
    if name in labels['name'].values:
        label = labels[labels['name'] == name]['class'].values[0]
        X.append(feature.flatten())
        y.append(label)
    # else:
    #     print(name)

In [225]:
# count each sample in y_train
from collections import Counter
counter = Counter(y)

print(counter)
# counter = Counter(X)
#
# print(counter)
print(len(X))


Counter({np.int64(3): 138, np.int64(1): 88, np.int64(2): 77, np.int64(4): 63, np.int64(0): 48})
414


In [226]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=None,test_size=0.2, stratify=y
)

In [227]:
print(len(X_train))

331


In [228]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
#
# # Step 1: Standardize
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
#
# # Step 2: Apply PCA
# pca = PCA(n_components=0.98)
# X_train_pca = pca.fit_transform(X_train_scaled)
# X_test_pca = pca.transform(X_test_scaled)
#
# print(f"Number of components retained: {pca.n_components_}")


In [229]:
from sklearn.model_selection import LeaveOneOut, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Build a single pipeline that does scaling → PCA → OVR-SVM
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca',    PCA(n_components=100)),
    ('clf',    OneVsRestClassifier(
                   SVC(kernel='rbf', probability=True, random_state=42, gamma = 0.0001 , C=10, class_weight='balanced')
               ))
])

# Leave-One-Out cross-validation
loo = LeaveOneOut()

# This will train on N-1 and predict on the held-out sample for each of the N folds
y_pred = cross_val_predict(pipeline, X, y, cv=loo, n_jobs=-1)

# Evaluate overall
print("LOOCV Accuracy:", accuracy_score(y, y_pred))

print("\nClassification Report:\n", classification_report(y, y_pred))

LOOCV Accuracy: 0.357487922705314

Classification Report:
               precision    recall  f1-score   support

           0       0.16      0.25      0.20        48
           1       0.32      0.27      0.29        88
           2       0.34      0.43      0.38        77
           3       0.40      0.28      0.33       138
           4       0.56      0.63      0.60        63

    accuracy                           0.36       414
   macro avg       0.36      0.37      0.36       414
weighted avg       0.37      0.36      0.36       414



In [230]:
from sklearn.model_selection import LeaveOneOut, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Build pipeline: StandardScaler → PCA → OVR-SVM
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=50)),
    ('clf', OneVsRestClassifier(
        SVC(kernel='rbf', probability=True, random_state=42, gamma=0.0001, C=10, class_weight='balanced')
    ))
])

# Fit on training set
pipeline.fit(X_train, y_train)

# Predict on test set
y_pred = pipeline.predict(X_test)

# Evaluation
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Test Accuracy: 0.3493975903614458

Classification Report:
               precision    recall  f1-score   support

           0       0.27      0.44      0.33         9
           1       0.15      0.11      0.13        18
           2       0.39      0.47      0.42        15
           3       0.24      0.18      0.20        28
           4       0.69      0.85      0.76        13

    accuracy                           0.35        83
   macro avg       0.35      0.41      0.37        83
weighted avg       0.32      0.35      0.33        83



In [52]:
from sklearn.model_selection import LeaveOneOut, GridSearchCV, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 1) Rebuild your base pipeline (no hard-coded params here)
base_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca',    PCA(n_components=120)),  # we’ll search over n_components
    ('clf',    OneVsRestClassifier(
                   SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)
               ))
])

# 2) Define the grid of hyperparameters
param_grid = {
    # 'pca__n_components': [20],          # try a few dims
    'clf__estimator__C': [0.01, 0.1, 0.5, 1.0, 5.0, 10, 100],       # regularization strengths
    'clf__estimator__gamma': [0.001, 0.005, 0.01, 0.05, 0.1],   # kernel widths
}

# 3) Set up Leave-One-Out CV
loo = LeaveOneOut()

# 4) Create the GridSearchCV
grid = GridSearchCV(
    base_pipeline,
    param_grid,
    cv=loo,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=2
)

# 5) Run the grid search on your full dataset (X, y)
grid.fit(X, y)

# 6) Inspect the best parameters & LOOCV score
print("Best hyperparameters:")
print(grid.best_params_)
print(f"Best LOOCV accuracy: {grid.best_score_:.4f}")

# 7) (Optional) Get detailed per-class results via a fresh LOOCV prediction
best_pipe = grid.best_estimator_
y_pred = cross_val_predict(best_pipe, X, y, cv=loo, n_jobs=-1)
print("\nClassification Report with best params:")
print(classification_report(y, y_pred))


Fitting 513 folds for each of 35 candidates, totalling 17955 fits
[CV] END clf__estimator__C=0.01, clf__estimator__gamma=0.001; total time=   1.3s
[CV] END clf__estimator__C=0.01, clf__estimator__gamma=0.001; total time=   1.2s
[CV] END clf__estimator__C=0.01, clf__estimator__gamma=0.001; total time=   1.2s
[CV] END clf__estimator__C=0.01, clf__estimator__gamma=0.001; total time=   1.1s
[CV] END clf__estimator__C=0.01, clf__estimator__gamma=0.001; total time=   1.1s
[CV] END clf__estimator__C=0.01, clf__estimator__gamma=0.001; total time=   1.7s
[CV] END clf__estimator__C=0.01, clf__estimator__gamma=0.001; total time=   1.1s
[CV] END clf__estimator__C=0.01, clf__estimator__gamma=0.001; total time=   1.1s
[CV] END clf__estimator__C=0.01, clf__estimator__gamma=0.001; total time=   1.2s
[CV] END clf__estimator__C=0.01, clf__estimator__gamma=0.001; total time=   1.1s
[CV] END clf__estimator__C=0.01, clf__estimator__gamma=0.001; total time=   1.0s
[CV] END clf__estimator__C=0.01, clf__estim

/home/mhd-kamal-rushdi/PycharmProjects/SimpleMeshClassifer/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mhd-kamal-rushdi/PycharmProjects/SimpleMeshClassifer/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mhd-kamal-rushdi/PycharmProjects/SimpleMeshClassifer/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisi

In [ ]:
print(len(y_pred))

In [173]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class SimpleANN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SimpleANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            nn.Linear(16, num_classes)
        )

    def forward(self, x):
        return self.model(x)


In [232]:
# add imbalance weights

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# DataLoaders
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)

# Model setup
input_dim = len(X[1])
print(input_dim)
num_classes = len(set(y))
model = SimpleANN(input_dim, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)


criterion = nn.CrossEntropyLoss(weight=class_weights)
# add weight decay and augmentation
optimizer = optim.Adam(model.parameters(), lr=0.0001)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Training loop
num_epochs = 870
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == targets).sum().item()
        total_train += targets.size(0)

    train_accuracy = correct_train / total_train

    # Evaluate on test set
    model.eval()
    test_loss = 0.0
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct_test += (predicted == targets).sum().item()
            total_test += targets.size(0)

    test_accuracy = correct_test / total_test
    avg_train_loss = total_loss / len(train_loader)
    avg_test_loss = test_loss / len(test_loader)

    print(f"Epoch {epoch+1}: "
          f"Train Loss = {avg_train_loss:.4f}, "
          f"Train Acc = {train_accuracy:.4f}, "
          f"Test Loss = {avg_test_loss:.4f}, "
          f"Test Acc = {test_accuracy:.4f}")

510
Epoch 1: Train Loss = 1.6236, Train Acc = 0.1511, Test Loss = 1.6131, Test Acc = 0.1084
Epoch 2: Train Loss = 1.6215, Train Acc = 0.1390, Test Loss = 1.6119, Test Acc = 0.1084
Epoch 3: Train Loss = 1.6080, Train Acc = 0.1571, Test Loss = 1.6136, Test Acc = 0.1084
Epoch 4: Train Loss = 1.6010, Train Acc = 0.1752, Test Loss = 1.6152, Test Acc = 0.1084
Epoch 5: Train Loss = 1.6106, Train Acc = 0.1601, Test Loss = 1.6143, Test Acc = 0.1084
Epoch 6: Train Loss = 1.6070, Train Acc = 0.1752, Test Loss = 1.6113, Test Acc = 0.1084
Epoch 7: Train Loss = 1.6031, Train Acc = 0.1934, Test Loss = 1.6098, Test Acc = 0.1205
Epoch 8: Train Loss = 1.6115, Train Acc = 0.1299, Test Loss = 1.6093, Test Acc = 0.1566
Epoch 9: Train Loss = 1.5997, Train Acc = 0.1511, Test Loss = 1.6087, Test Acc = 0.1566
Epoch 10: Train Loss = 1.5985, Train Acc = 0.1722, Test Loss = 1.6075, Test Acc = 0.1687
Epoch 11: Train Loss = 1.6080, Train Acc = 0.1813, Test Loss = 1.6066, Test Acc = 0.1687
Epoch 12: Train Loss = 1.6

In [26]:
import torch
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(16 * 17 * 10, 64),  # assuming shape is (3, 17, 10)
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        # x is expected to be of shape (B, 3, 17, 10)
        return self.model(x)


In [188]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.utils.class_weight import compute_class_weight

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).view(-1, 3, 17, 10)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).view(-1, 3, 17, 10)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# DataLoaders
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)

# Model setup
num_classes = len(set(y))
model = SimpleCNN(num_classes)  # assumes you defined SimpleCNN as before
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Class weights for imbalance
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # weight decay added

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        # --- Optional: Add data augmentation here (e.g., random flip or noise) ---
        # inputs = augment(inputs)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == targets).sum().item()
        total_train += targets.size(0)

    train_accuracy = correct_train / total_train

    # Evaluation
    model.eval()
    test_loss = 0.0
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct_test += (predicted == targets).sum().item()
            total_test += targets.size(0)

    test_accuracy = correct_test / total_test
    avg_train_loss = total_loss / len(train_loader)
    avg_test_loss = test_loss / len(test_loader)

    print(f"Epoch {epoch+1}: "
          f"Train Loss = {avg_train_loss:.4f}, "
          f"Train Acc = {train_accuracy:.4f}, "
          f"Test Loss = {avg_test_loss:.4f}, "
          f"Test Acc = {test_accuracy:.4f}")


AssertionError: Size mismatch between tensors

In [ ]:
len(X_test[0])

In [ ]:
# count each sample in y_train
from collections import Counter
counter = Counter(y_train)

print(counter)
counter = Counter(y_test)

print(counter)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Create datasets

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        feature = feature.view(3, 170)  # RESHAPE here!
        return feature, self.labels[idx]

train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

# Create loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

class Normals1DCNN(nn.Module):
    def __init__(self, num_classes):
        super(Normals1DCNN, self).__init__()
        self.conv1 = nn.Conv1d(3, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.drop1 = nn.Dropout(0.2)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.drop2 = nn.Dropout(0.2)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(128)
        self.drop3 = nn.Dropout(0.3)

        self.pool = nn.MaxPool1d(2)
        self.global_pool = nn.AdaptiveAvgPool1d(1)

        self.drop_fc = nn.Dropout(0.4)
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.drop1(x)
        x = self.pool(x)

        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.drop2(x)
        x = self.pool(x)

        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.drop3(x)
        x = self.pool(x)

        x = self.global_pool(x)
        x = x.squeeze(-1)

        x = self.drop_fc(x)
        x = self.fc(x)
        return x

In [ ]:
print(train_dataset.__getitem__(1)[0].shape)

In [ ]:
for input, target in train_loader:
    print(input.shape)
    print(target.shape)
    break

In [ ]:
print(len(set(y)))

In [ ]:
# Model, loss, optimizer
num_classes = len(set(y))  # Total number of classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Normals1DCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 200

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)

    # --- Test loss added here ---
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
    avg_test_loss = test_loss / len(test_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Test Loss: {avg_test_loss:.4f}")


In [ ]:
# flush the memory cell
print(torch.cuda.memory_allocated())
del  loss, optimizer, model, targets, inputs, outputs

print(torch.cuda.memory_allocated())  # Current memory in use

import gc
# Force garbage collection
gc.collect()

# Clear PyTorch's cache
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated())  # Current memory in use

In [ ]:
# Evaluation
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(targets.cpu().numpy())

test_accuracy = accuracy_score(y_true, y_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
from collections import Counter
from sklearn.metrics import accuracy_score

# Evaluation
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(targets.cpu().numpy())

# Accuracy
test_accuracy = accuracy_score(y_true, y_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Count predictions per class
pred_counter = Counter(y_pred)

print("\nPredicted class distribution (regardless of correctness):")
for label, count in sorted(pred_counter.items()):
    print(f"  Class {label}: {count} predictions")


In [ ]:
#
# Top-2 evaluation
model.eval()
correct_top2 = 0
total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)

        # Get top-2 predictions per sample
        top2 = torch.topk(outputs, k=2, dim=1).indices  # shape: (batch_size, 2)

        # Compare: is true label in top 2 predictions?
        for i in range(targets.size(0)):
            if targets[i] in top2[i]:
                correct_top2 += 1
            total += 1

top2_accuracy = correct_top2 / total
print(f"Top-2 Accuracy: {top2_accuracy:.4f}")

